In [ ]:
Counts = {}
tokens = ['He', 'skipped', 'off', 'the', 'gunrest', 'and', 'looked', 'gravely', 'at', 'his', 'watcher', 'gathering', 'about', 'his', 'legs', 'the', 'loose', 'folds', 'of', 'his', 'gown']
ngram_size = 4
    

In [ ]:
def ngrams_from_tokens_sen(tokens, ngram_size, Counts):
    assert(ngram_size == Counts["NgramSize"])
    assert(type(ngram_size) == int and ngram_size > 0)
    ngram = ["w" for i in range(ngram_size)]
    for i in range(len(tokens)):
        for j in range(ngram_size - 1):
            ngram[j] = ngram[j + 1]
        ngram[ngram_size - 1] = tokens[i]
        
        history = tuple((ngram[i] for i in range(ngram_size - 1)))
        word = ngram[ngram_size - 1]

        if history in Counts.keys():
            if word in Counts[history].keys():
                Counts[history][word] += 1
            else:
                Counts[history][word] = 1
        else:
            Counts[history] = {}
            Counts[history][word] = 1


def unigram_from_tokens_sen(tokens, Counts):
    assert(Counts["NgramSize"] == 1)
    for token in tokens:
        if token in Counts.keys():
            Counts[token] += 1
        else:
            Counts[token] = 1


def sum_values(Counts):
    SumVal = {}
    SumVal["NgramSize"] == Counts["NgramSize"]
    if(SumVal["NgramSize"] == 1):
        SumVal["SUM"] = sum(Counts.values())
    else:
        for history in Counts.keys():
            SumVal[history] = sum(Counts[history].values())
    return SumVal


# extract from bigram
def unigram_keser_nay_data(Counts):
    assert(Counts["NgramSize"] == 2)
    keser_nay_unigram = {}
    total_unique_bigrams = 0
    for history in Counts.keys():
        total_unique_bigrams += len(Counts[history].keys())
    keser_nay_unigram["total_bigrams"] = total_unique_bigrams
    for history in Counts.keys():
        for word in Counts[history].keys():
            if word in keser_nay_unigram.keys():
                keser_nay_unigram[word] += 1
            else:
                keser_nay_unigram[word] = 1
    return keser_nay_unigram
    
                
def conditional_prob(history, word, Counts, SumCounts):
    assert(len(history) + 1 == Counts["NgramSize"])
    assert(Counts["NgramSize"] == SumCounts["NgramSize"])
    if history in Counts.keys():
        if word in Counts[history].keys():
            return Counts[history][word] / SumCounts[history]
        else:
            return 0
    else:
        return 0
    

def unigram_prob(word, Counts, SumCounts):
    assert(Counts["NgramSize"] == 1)
    assert(SumCounts["NgramSize"] == 1)
    if word in Counts.keys():
        return Counts[word] / SumCounts["SUM"]
    else:
        return 0

In [ ]:
ngrams_from_tokens_sen(tokens, ngram_size, Counts)
print(Counts)

In [8]:
def witten_bell_conditional(history, word, Counts_all, Sums_all, N):
    assert(type(N) == int and N > 0)
    assert(Counts_all[N]["NgramSize"] == N)
    assert(Sums_all[N]["NgramSize"] == N)
    assert(len(history) + 1 == N)
    
    # Base case
    if N == 1:
        return unigram_prob(word, Counts_all[N], Sums_all[N])
    else:
        if history in Counts_all[N].keys():
            novel_history_words = len(Counts_all[N][history].keys())  #time compleixty O(1)
            accourences_history = Sums_all[N][history].values()
            prob_word_novel = novel_history_words / (accourences_history + novel_history_words)

            if word in Counts_all[N][history].keys():
                prob_cur = (1 - prob_word_novel) * Counts_all[N][history][word] / Sums_all[N][history]
            else:
                prob_cur = 0
                
            prob_recur = prob_word_novel * witten_bell_conditional(history[1:], word, Counts_all, Sums_all, N - 1)
            return prob_cur + prob_recur
        else:
            return 0


In [ ]:
def keser_ney(history, word, Counts_all, Sums_all, kenser_uni ,N, d):
    assert(d >= 0 and d <= 1)
    assert(type(N) == int and N > 0)
    assert(Counts_all[N]["NgramSize"] == N)
    assert(Sums_all[N]["NgramSize"] == N)
    assert(len(history) + 1 == N)
    

    # Base case
    if N == 1:
        if word in kenser_uni.keys():
            return kenser_uni[word] / kenser_uni["total_bigrams"]
        else:
            return 0
    else:
        if history in Counts_all[N].keys():
            novel_history_words = len(Counts_all[N][history].keys())  #time compleixty O(1)
            occourences_history = Sums_all[N][history].values()
            rec_weight = d * (novel_history_words / occourences_history)
            
            if word in Counts_all[N][history].keys():
                cur_prob = max(0, Counts_all[N][history][word] - d) / occourences_history
            else:
                cur_prob = 0

            recur_prob = keser_ney(history[1:], word, Counts_all, Sums_all, kenser_uni,N - 1, d)
            return rec_weight * recur_prob + cur_prob
        else:
            return 0

In [ ]:
def likehood_of_sen_tokens(tokens, ngram_size, Counts_all, Sums_all, keser_uni, d, method):
    assert(len(tokens) > 0)
    likelyhood = 1
    ngram = ["w" for i in range(ngram_size)]
    for i in range(len(tokens)):
        for j in range(ngram_size - 1):
            ngram[j] = ngram[j + 1]
        ngram[ngram_size - 1] = tokens[i]
        
        history = tuple((ngram[i] for i in range(ngram_size - 1)))
        word = ngram[ngram_size - 1]

        if method == "witten_bell":
            likelyhood *= witten_bell_conditional(history, word, Counts_all, Sums_all, ngram_size)
        elif method == "keser_ney":
            likelyhood *= keser_ney(history, word, Counts_all, Sums_all, keser_uni, ngram_size, d)
        else:
            assert(False)
    return likelyhood